In [1]:
##### REQUIRES THE DATAFRAME FOLDER TO BE NAMED 'Cohorts', WHICH INCLUDES ALL PRECOMPUTED DATAFRAMES #####
import fiber
from fiber.cohort import Cohort
from fiber.condition import Patient, MRNs
from fiber.condition import Diagnosis
from fiber.condition import Measurement, Encounter, Drug
from fiber.storage import yaml as fiberyaml
import pandas as pd
import pyarrow.parquet as pq
import numpy as np
import os
from functools import reduce 

DB Password: ········


In [2]:
#load Cohorts
Case_filtered_15_540 = pq.read_table('Cohorts/Case/Case_filtered_15_540.parquet').to_pandas()
Control_filtered_15_540 = pq.read_table('Cohorts/Control/Control_filtered_15_540.parquet').to_pandas()

In [3]:
#generic function get cohort 
def df_to_cohort(df):
    mrns = list(df.index.values)
    condition = MRNs(mrns)
    return Cohort(condition)

In [4]:
#condition= FIBER Condition 
#df_mrn = pandas Data frame with Patient MRNS 
#name_df = Name o the new Dataframe
#name_feature =Feature name 
#frequency = type of occurence: EVER, COUNT,WINDOW
#Cohort_type = Case/Control
def get_has_certain_condition(condition, df_mrn, name_feature, gap_in_days,window_in_days, frequency, cohort_type):
    #get cohort
    Onset_column = "HT_Onset" if cohort_type == "Case" else "last_encounter"
    cohort = df_to_cohort(df_mrn)
    #get cohort with condition
    cohort_condition = cohort.get(condition)
    # Prefilter condition frame, so there are only the rows of interest --> rows which fulfill the gap condition
    try:
        Filtered_DF = df_mrn.merge(cohort_condition, left_index=True, right_on = "medical_record_number")
    except:
        print("DataFrame is empty. Condition does not fit to any record!")
        raise SystemExit(0)
    Filtered_DF = Filtered_DF.set_index("medical_record_number")
    # Use only necessary columns age_in_days + HT_Onset
    Filtered_DF = Filtered_DF[["age_in_days", Onset_column]]
    Filtered_DF = Filtered_DF.loc[(Filtered_DF[Onset_column] - Filtered_DF["age_in_days"]) >= gap_in_days]
    # Filtered_DF contains all gap relevant entries
    
    # get MRNs of input file 
    #cohort_indexes = df_mrn.index
    #get 0 for no diagnosis and 1 for has diagnosis
    cohort_mrn_diagnosis=[]
    if frequency=='EVER':
        #Remove all duplicate MRNs (index)
        Filtered_DF = Filtered_DF.loc[~Filtered_DF.index.duplicated(keep='first')]
        # Set all Rows to 1, because they occur in the dataframe
        Filtered_DF["has_condition_" + name_feature] = 1
        # Prepare merging to the whole cohort
        to_merge = Filtered_DF["has_condition_" + name_feature]
        to_merge = to_merge.to_frame()
        #Merge it with a left outer join
        cohort_mrn_diagnosis = df_mrn.merge(to_merge, left_index = True, right_index = True, how="left")
        # Fill all Rows, which did not appear in the condition with 0
        cohort_mrn_diagnosis["has_condition_" + name_feature].fillna(0, inplace=True)
    elif frequency=='COUNT':
        # Group by Index (MRNs)
        Count_DF = Filtered_DF.groupby([Filtered_DF.index]).count()
        # Count random column, does not matter which
        Count_DF = Count_DF[Onset_column]
        #Convert Series to Frame
        Count_DF = Count_DF.to_frame()
        # Rename column
        Count_DF.rename(columns={Onset_column:'number_of_occurences_' + name_feature},inplace=True)
        # Merge as above
        cohort_mrn_diagnosis = df_mrn.merge(Count_DF, left_index = True, right_index = True, how="left")
        cohort_mrn_diagnosis["number_of_occurences_" + name_feature].fillna(0, inplace=True)
    elif frequency=='WINDOW':
        #max window size
        Filtered_DF = Filtered_DF.loc[(Filtered_DF[Onset_column] - Filtered_DF["age_in_days"]) <= (window_in_days+gap_in_days)]
        # Group by Index (MRNs)
        Count_DF = Filtered_DF.groupby([Filtered_DF.index]).count()
        # Count random column, does not matter which
        Count_DF = Count_DF[Onset_column]
        #Convert Series to Frame
        Count_DF = Count_DF.to_frame()
        # Rename column
        Count_DF.rename(columns={Onset_column:'number_of_occurences_' + name_feature},inplace=True)
        # Merge as above
        cohort_mrn_diagnosis = df_mrn.merge(Count_DF, left_index = True, right_index = True, how="left")
        cohort_mrn_diagnosis["number_of_occurences_" + name_feature].fillna(0, inplace=True)
    else:
        print("function not available")
    #convert list cohort_mrn_diagnosis to panda
    #condition_name= 'has_condition_'+name_feature
    #col_names = ['medical_record_number',condition_name]
    #df_final=pd.DataFrame(cohort_mrn_diagnosis,columns=col_names)
    #df_final.set_index('medical_record_number', inplace=True)
    #df_final=df_mrn.merge( df_final, left_index=True, right_index=True)
    #df_final.to_parquet(name_df)
    #print(df_final)
    df_final = cohort_mrn_diagnosis
    return df_final

In [5]:
case = df_to_cohort(Case_filtered_15_540)
control = df_to_cohort(Control_filtered_15_540)

In [6]:
Med_CV_condition = (Drug('%Lisinopril%') | 
                    Drug('%Hydrochlorothiazide%')|
                    Drug('%Metoprolol tartrate%')|
                    Drug('%Metoprolol succinate%')|
                    Drug('%Amlodipine besylate%')|
                    Drug('%Atenolol%')|
                    Drug('%Losartan potassium%')|
                    Drug('%Enalapril maleate%')
                    )


In [7]:
##### CASES ######

In [8]:
Med_CV_window = get_has_certain_condition(Med_CV_condition, Case_filtered_15_540, "Med_CV", 180 ,720, "WINDOW", "Case")

Fetching data for Drug (...)


In [10]:
Med_CV_window.loc[Med_CV_window['number_of_occurences_Med_CV'] >=1]

,number_of_encounters,Age_BP_condition,Age_ICD_condition,Age_MED_condition,HT_Onset,Earliest_Condition,first_encounter,total_record_time,year_of_birth,race,religion,gender,number_of_occurences_Med_CV
medical_record_number,,,,,,,,,,,,,
1039156031,89,26224.0,NaN,NaN,26224.0,Age_BP_condition,24948,1276.0,1946,Unknown,Christian,Female,3.0
1053244405,136,23046.0,23121.0,NaN,23046.0,Age_BP_condition,17352,5694.0,1952,Other,Catholic,Female,1.0
1389146005,30,23341.0,NaN,NaN,23341.0,Age_BP_condition,21927,1414.0,1954,Unknown,Unknown,Female,6.0
1445231984,20,19062.0,19062.0,NaN,19062.0,Age_BP_condition,18202,860.0,1964,Unknown,,Female,1.0
1520232832,730,26096.0,NaN,NaN,26096.0,Age_BP_condition,23342,2754.0,1936,Ba,Protestant,Female,4.0
1573256350,183,29383.0,29341.0,NaN,29341.0,Age_ICD_condition,26310,3031.0,1935,White,Catholic,Male,1.0
1787729063,99,16991.0,16991.0,NaN,16991.0,Age_BP_condition,13945,3046.0,1968,White,Unknown,Male,1.0
1829325469,214,27315.0,27628.0,NaN,27315.0,Age_BP_condition,26207,1108.0,1938,White,,Male,6.0
1872606088,102,10410.0,NaN,NaN,10410.0,Age_BP_condition,5365,5045.0,1989,White,Jewish,Male,2.0


In [11]:
Med_CV_window.to_parquet('med_cv_window_2yr_case.parquet')

In [12]:
##### CONTROLS######

In [13]:
Med_CV_window = get_has_certain_condition(Med_CV_condition, Control_filtered_15_540, "Med_CV", 180 ,720, "WINDOW", "Control")

Fetching data for Drug (...)


In [14]:
Med_CV_window.loc[Med_CV_window['number_of_occurences_Med_CV'] >=1]

,last_encounter,number_of_encounters,first_encounter,total_record_time,year_of_birth,race,religion,gender,number_of_occurences_Med_CV
medical_record_number,,,,,,,,,
3178155540,23319,34,20952,2367,1954,White,None,Female,2.0
3289795948,21567,34,19973,1594,1953,White,Jewish,Female,1.0
3978386652,30357,51,27582,2775,1931,White,Christian,Female,1.0
97804271,19312,42,17837,1475,1964,Unknown,Unknown,Male,1.0
990540085,20808,101,18879,1929,1961,White,Jewish,Male,1.0
1013780962,23584,60,19040,4544,1952,White,Episcopal,Female,1.0
1317259415,22172,33,20050,2122,1957,Other,Catholic,Female,1.0
150942057,17524,68,16804,720,1970,Unknown,Catholic,Male,1.0
2266422359,17014,18,16290,724,1964,White,,Female,1.0


In [15]:
Med_CV_window.to_parquet('med_cv_window_2yr_control.parquet')